In [ ]:
# 2020-10-28 created by Akson

In [ ]:
# Code 14.1

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image

# load image
china = load_sample_image('china.jpg') / 255
flower = load_sample_image('flower.jpg') / 255
images = np.array([china, flower])
batch_size, height, width, channels = images.shape

# create 2 filters
filters = np.zeros(shape = (7, 7, channels, 2), dtype = np.float32)
filters[:, 3, :, 0] = 1
filters[3, :, :, 1] = 1
print(filters.shape)

outputs = tf.nn.conv2d(images, filters, strides = 1, padding = 'SAME')

plt.imshow(outputs[0, :, :, 1], cmap = 'gray')
plt.show()
plt.imshow(outputs[1, :, :, 1], cmap = 'gray')
plt.show()

In [ ]:
# Code 14.2
# 画出滤波器

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")
    
plot_image(filters[:, :, 0, 0])
plt.show()
plot_image(filters[:, :, 0, 1])
plt.show()

In [ ]:
# Code14.3
# 加载Fashion MNIST数据集

import tensorflow.keras as keras

# 加载全部数据
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 将一部分训练集作为验证集
X_train = X_train_full[:-5000]
X_valid = X_train_full[-5000:]
y_train = y_train_full[:-5000]
y_valid = y_train_full[-5000:]
# print(X_train.shape)

# 标准化
X_mean = X_train.mean(axis = 0, keepdims = True)
#print(X_mean.shape)
X_std = X_train.std(axis = 0, keepdims = True)

X_train = (X_train - X_mean) / (X_std + 1e-7)
X_valid = (X_valid - X_mean) / (X_std + 1e-7)
X_test = (X_test - X_mean) / (X_std + 1e-7)

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
# Code14.4
# 建立模型来训练

model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation = 'relu', padding = 'same', input_shape = [28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same'),
    keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same'),
    keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

y_pred = model.predict(X_test[:10])
print(y_test[:10])
print(y_pred)

In [ ]:
# Code14.5
# ResNet-34

# create RU
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides = 1, activation = 'relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides = strides, padding = 'same', use_bias = False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides = 1, padding = 'same', use_bias = False),
            keras.layers.BatchNormalization()
        ]
        
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides = strides, padding = 'same', use_bias = False),
                keras.layers.BatchNormalization()
            ]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
# Code14.6
# create ResNet-34

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides = 2, input_shape = [224, 224, 3], padding = 'same', use_bias = False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = 'same'))
prev_filters = 64

# add RUs
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides = strides))
    pre_filters = filters

model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [ ]:
# Code14.7
# 使用现成的模型

model = keras.applications.resnet50.ResNet50(weights = 'imagenet')

# 改变图片大小
images_resized = tf.image.resize(images, [224, 224])
inputs = keras.applications.resnet50.preprocess_input(images_resized * 255)
Y_proba = model.predict(inputs)

topK = keras.applications.resnet50.decode_predictions(Y_proba, top = 3)
for image_index in range(len(images)):
    print('Image #{}'.format(image_index))
    for class_id, name, y_proba in topK[image_index]:
        print('  {} - {:12s}{:.2f}'.format(class_id, name, y_proba * 100))

In [ ]:
# Code14.8
# loadDataset

import tensorflow_datasets as tfds

# 在Mac平台用这个
# dataset, info = tfds.load('tf_flowers', data_dir = '../dataset/', as_supervised = True, with_info = True)

# 在windows平台上用这个
# dataset, info = tfds.load('tf_flowers', as_supervised = True, with_info = True)

#  在colab上用这个
dataset, info = tfds.load('tf_flowers', as_supervised = True, with_info = True, try_gcs = True)

dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes

print('dataset_size: ', dataset_size)
print('class_names: ', class_names)
print('n_classes: ', n_classes)

In [ ]:
# Code14.9
# 拆分训练数据集

# 在Mac平台用这个
# test_set = tfds.load('tf_flowers', split = 'train[:10%]', data_dir = '../dataset/', as_supervised = True)
# valid_set = tfds.load('tf_flowers', split = 'train[10%:25%]', data_dir = '../dataset/', as_supervised = True)
# train_set = tfds.load('tf_flowers', split = 'train[25%:]', data_dir = '../dataset/', as_supervised = True)

# 在windows平台上用这个
# test_set = tfds.load('tf_flowers', split = 'train[:10%]', as_supervised = True)
# valid_set = tfds.load('tf_flowers', split = 'train[10%:25%]', as_supervised = True)
# train_set = tfds.load('tf_flowers', split = 'train[25%:]', as_supervised = True)

#  在colab上用这个
test_set = tfds.load('tf_flowers', split = 'train[:10%]', as_supervised = True, try_gcs = True)
valid_set = tfds.load('tf_flowers', split = 'train[10%:25%]', as_supervised = True, try_gcs = True)
train_set = tfds.load('tf_flowers', split = 'train[25%:]', as_supervised = True, try_gcs = True)

In [ ]:
# Code14.10
# preprocess

def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label
    
batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
# Code14.11
# load model and train

base_model = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation = 'softmax')(avg)
model = keras.Model(inputs = base_model.input, outputs = output)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(lr = 0.2, momentum = 0.9, decay = 0.01)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
history = model.fit(train_set, epochs = 5, validation_data = valid_set)

In [ ]:
# Code14.12
# 

for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9, nesterov = True, decay = 0.001)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
history = model.fit(train_set, steps_per_epoch = int(0.75 * dataset_size / batch_size), validation_data = valid_set, validation_steps = int(0.15 * dataset_size / batch_size), epochs = 40)